# XGBoost 极端梯度提升

## 算法原理

XGBoost (eXtreme Gradient Boosting) 是梯度提升算法的高效实现，具有以下特点：

1. **正则化**: 在目标函数中加入 L1/L2 正则化项，防止过拟合
2. **二阶导数**: 使用二阶泰勒展开，比传统梯度提升更精确
3. **并行化**: 特征级别的并行计算
4. **缺失值处理**: 自动学习缺失值的最优处理方向

## 目标函数

$$\mathcal{L}(\phi) = \sum_{i} l(y_i, \hat{y}_i) + \sum_{k} \Omega(f_k)$$

其中 $\Omega(f) = \gamma T + \frac{1}{2}\lambda ||w||^2$ 是正则化项。

## 与传统 GBDT 的区别

| 特性 | 传统 GBDT | XGBoost |
|------|-----------|----------|
| 优化 | 一阶导数 | 二阶导数 |
| 正则化 | 无 | L1 + L2 |
| 并行化 | 不支持 | 特征级并行 |
| 缺失值 | 需预处理 | 自动处理 |

In [ ]:
# 导入必要的库
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

使用鸢尾花数据集进行多分类任务演示。

In [ ]:
# 加载数据
data = load_iris()
X, y = data.data, data.target

# 三分数据集：训练集、验证集、测试集
# 第一步：划分训练集和临时集（8:2）
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# 第二步：将临时集划分为验证集和测试集（1:1）
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp
)

print(f"训练集: {X_train.shape[0]} 样本")
print(f"验证集: {X_val.shape[0]} 样本")
print(f"测试集: {X_test.shape[0]} 样本")

## 2. XGBoost 分类器

使用 XGBClassifier 进行训练，并在验证集上监控性能。

In [ ]:
# 创建 XGBoost 分类器
xgb_clf = xgb.XGBClassifier(
    n_estimators=100,          # 树的数量
    max_depth=3,               # 树的最大深度
    learning_rate=0.1,         # 学习率
    subsample=0.8,             # 每棵树使用的样本比例
    colsample_bytree=0.8,      # 每棵树使用的特征比例
    reg_alpha=0,               # L1 正则化系数
    reg_lambda=1,              # L2 正则化系数
    random_state=RANDOM_STATE,
    eval_metric='mlogloss'     # 评估指标
)

# 训练模型，使用验证集进行早停
xgb_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False              # 不打印每轮日志
)

# 评估
y_pred = xgb_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"测试集准确率: {test_accuracy:.4f}")

## 3. 早停策略

使用早停可以防止过拟合并节省训练时间。

In [ ]:
# 使用早停的 XGBoost
xgb_early = xgb.XGBClassifier(
    n_estimators=500,          # 设置较大的值
    max_depth=3,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    eval_metric='mlogloss',
    early_stopping_rounds=10   # 10轮无改善则停止
)

# 训练
xgb_early.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print(f"最佳迭代次数: {xgb_early.best_iteration}")
print(f"最佳验证分数: {xgb_early.best_score:.4f}")

# 评估
y_pred_early = xgb_early.predict(X_test)
early_accuracy = accuracy_score(y_test, y_pred_early)
print(f"测试集准确率: {early_accuracy:.4f}")

## 4. 特征重要性

In [ ]:
# 获取特征重要性
feature_names = data.feature_names
importances = xgb_clf.feature_importances_

# 按重要性排序
indices = np.argsort(importances)[::-1]

print("特征重要性排序:")
for i in indices:
    print(f"  {feature_names[i]}: {importances[i]:.4f}")

## 5. 交叉验证评估

In [ ]:
# 交叉验证
cv_clf = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=RANDOM_STATE
)

cv_scores = cross_val_score(cv_clf, X, y, cv=5, scoring='accuracy')

print(f"交叉验证分数: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 6. 参数调优建议

XGBoost 的关键参数及其调优策略：

In [ ]:
# 不同学习率的影响
learning_rates = [0.01, 0.1, 0.3, 0.5]

print("学习率对比:")
print(f"{'学习率':>10} {'训练准确率':>12} {'测试准确率':>12}")
print("-" * 36)

for lr in learning_rates:
    clf = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=lr,
        max_depth=3,
        random_state=RANDOM_STATE
    )
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    
    print(f"{lr:>10.2f} {train_acc:>12.4f} {test_acc:>12.4f}")

## 7. 单元测试验证

In [ ]:
def test_xgboost():
    """XGBoost 功能测试"""
    
    # 测试1: 模型应该正确训练
    assert hasattr(xgb_clf, 'feature_importances_'), "模型未正确训练"
    
    # 测试2: 准确率应该在合理范围
    assert test_accuracy >= 0.8, f"测试集准确率过低: {test_accuracy}"
    
    # 测试3: 特征重要性应该存在
    assert len(importances) == X.shape[1], "特征重要性维度不正确"
    
    # 测试4: 早停应该正常工作
    assert xgb_early.best_iteration < 500, "早停未生效"
    
    # 测试5: 预测结果形状应该正确
    assert y_pred.shape == y_test.shape, "预测结果形状不正确"
    
    # 测试6: 交叉验证应该返回正确数量的分数
    assert len(cv_scores) == 5, "交叉验证折数不正确"
    
    print("所有测试通过!")

test_xgboost()

## 总结

### XGBoost 的优势

1. **高效**: 并行计算和缓存优化
2. **正则化**: 内置 L1/L2 正则化防止过拟合
3. **灵活**: 支持自定义目标函数和评估指标
4. **鲁棒**: 自动处理缺失值

### 调参建议

1. **首先调整**: `n_estimators` 和 `learning_rate`（互相关联）
2. **然后调整**: `max_depth`（通常 3-10）
3. **正则化**: `reg_alpha` 和 `reg_lambda`（防止过拟合）
4. **采样**: `subsample` 和 `colsample_bytree`（增加随机性）

### 常用参数组合

- 小数据集: `learning_rate=0.1`, `n_estimators=100-500`
- 大数据集: `learning_rate=0.01-0.05`, `n_estimators=1000+`, 配合早停